In [1]:
# dimensions of our images.
img_width, img_height = 224, 224
%matplotlib inline


train_data_dir = '/Users/shvadskd/src/MyProjects/DeepLearning/fast_ai/courses/deeplearning1/nbs/data/dogscats/train'
validation_data_dir = '/Users/shvadskd/src/MyProjects/DeepLearning/fast_ai/courses/deeplearning1/nbs/data/dogscats/valid'

In [2]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

img_path = train_data_dir + '/cats/cat.9999.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5005)


In [3]:

model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                                              Output Shape                                                                    Param #                       
input_1 (InputLayer)                                                                      (None, None, None, 3)                                                           0                             
________________________________________________________________________________________________________________________________________________________________________________________________________
block1_conv1 (Conv2D)                                                                     (None, None, None, 64)                                                          1792                      

Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
________________________________________________________________________________________________________________________________________________________________________________________________________


In [4]:
model.input

/input_1

In [17]:
from keras.preprocessing.image import ImageDataGenerator
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)


train_generator_bottleneck = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=8,
        class_mode=None,
        shuffle=False)

validation_generator_bottleneck = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=8,
        class_mode=None,
        shuffle=False)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [18]:
nb_epoch = 1
nb_train_samples = 23000
nb_validation_samples = 2000

In [21]:
bottleneck_features_train = model.predict_generator(train_generator_bottleneck, nb_train_samples)
np.save(open('models/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

MemoryError: Error allocating 4816896 bytes of device memory (out of memory).
Apply node that caused the error: GpuContiguous(GpuDimShuffle{0,3,1,2}.0)
Toposort index: 144
Inputs types: [CudaNdarrayType(float32, 4D)]
Inputs shapes: [(8, 3, 224, 224)]
Inputs strides: [(150528, 1, 672, 3)]
Inputs values: ['not shown']
Outputs clients: [[GpuDnnConv{algo='small', inplace=True}(GpuContiguous.0, GpuContiguous.0, GpuAllocEmpty.0, GpuDnnConvDesc{border_mode='half', subsample=(1, 1), conv_mode='conv', precision='float32'}.0, Constant{1.0}, Constant{0.0})]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [ ]:
bottleneck_features_validation = model.predict_generator(validation_generator_bottleneck, nb_validation_samples)
np.save(open('models/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

In [10]:
train_data = np.load(open('models/bottleneck_features_train.npy', 'rb'))
train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

validation_data = np.load(open('models/bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

In [16]:
print(train_data.shape)
print(train_labels.shape)
print(validation_data.shape)
print(validation_labels.shape)

(16384, 7, 7, 512)
(2048,)
(6656, 7, 7, 512)
(832,)


In [12]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation='sigmoid'))

model_top.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
nb_epoch=10

model_top.fit(train_data, train_labels,
          epochs=nb_epoch, batch_size=32,
          validation_data=(validation_data, validation_labels))


/Users/shvadskd/anaconda/envs/ztdl-gpu/lib/python3.5/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Input arrays should have the same number of samples as target arrays. Found 16384 input samples and 2048 target samples.